In [2]:
from keras.models import Model 
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import np_utils
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
import os
import datetime
from sklearn.model_selection import train_test_split

In [12]:
poor = pd.read_csv("SimData/bank_reserves_outputs_poor.csv", header=None)
middle = pd.read_csv("SimData/bank_reserves_outputs_middle.csv", header=None)
rich = pd.read_csv("SimData/bank_reserves_outputs_rich.csv", header=None)
sc = StandardScaler()
bank_reserves = []
for i in np.arange(0, poor.shape[0]):
    sample = pd.concat([poor.iloc[i], middle.iloc[i]], axis=0).T
    sample = pd.concat([sample, rich.iloc[i]], axis=0).T
    sample_std = sc.fit_transform(sample.to_frame())
    bank_reserves.append(sample_std)

In [24]:
ecv_active = pd.read_csv("SimData/epsteinCV_outputs_active.csv", header=None)
ecv_jailed = pd.read_csv("SimData/epsteinCV_outputs_jailed.csv", header=None)
ecv_quiet = pd.read_csv("SimData/epsteinCV_outputs_quiet.csv", header=None)
sc = StandardScaler()
ecv = []
for i in np.arange(0, ecv_active.shape[0]):
    sample = pd.concat([ecv_active.iloc[i], ecv_jailed.iloc[i]], axis=0).T
    sample = pd.concat([sample, ecv_quiet.iloc[i]], axis=0).T
    sample_std = sc.fit_transform(sample.to_frame())
    ecv.append(sample_std)

In [25]:
def import_ff_data(filename):
    expected_columns=155
    data = []
    with open(filename, 'r') as file:
        for line in file:
            row = line.strip().split(',')
            if len(row) < expected_columns:
                row += [np.nan] * (expected_columns - len(row))
            data.append(row)
    df = pd.DataFrame(data)
    def fill_last_valid(row):
        for i in range(1, len(row)):
            if pd.isna(row[i]):
                row[i] = row[i - 1]  
        return row
    df_filled = df.apply(fill_last_valid, axis=1)
    return df_filled

In [26]:
ff_onfire = import_ff_data("SimData/forest_fire_outputs_onfire.csv")
print("check 1")
ff_fine = import_ff_data("SimData/forest_fire_outputs_fine.csv")
print("check 2")
ff_burned = import_ff_data("SimData/forest_fire_outputs_burned.csv")
sc = StandardScaler()
ff = []
for i in np.arange(0, ff_onfire.shape[0]):
    sample = pd.concat([ff_onfire.iloc[i], ff_fine.iloc[i]], axis=0).T
    sample = pd.concat([sample, ff_burned.iloc[i]], axis=0).T
    sample_std = sc.fit_transform(sample.to_frame())
    ff.append(sample_std)

check 1
check 2


In [27]:
def DAE_reduction(df):
    df_flat = [sample.flatten() for sample in df]
    df_flat = np.asarray(df_flat)
    train, test = train_test_split(df_flat, test_size=0.20, random_state=42)
    input = Input(shape=(df_flat.shape[1],))

    encoded = Dense(30, activation='relu')(input)
    encoded = Dense(20, activation='relu')(encoded)
    encoded = Dense(10, activation='relu')(encoded)
    encoded = Dense(5, activation='relu')(encoded)

    decoded = Dense(10, activation='relu')(encoded)
    decoded = Dense(20, activation='relu')(decoded)
    decoded = Dense(30, activation='relu')(decoded)
    output = Dense(df_flat.shape[1], activation=None)(decoded)

    autoencoder = Model(input, output)
    autoencoder.summary()
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    
    autoencoder.fit(train, train,
     epochs=20,
     batch_size=256,
     shuffle=True,
     validation_data=(test, test))
    encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(index=4).output)
    encoded_ts = encoder.predict(df_flat)
    return encoded_ts

In [28]:
br_dae = DAE_reduction(bank_reserves)

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 303)]             0         
                                                                 
 dense_24 (Dense)            (None, 30)                9120      
                                                                 
 dense_25 (Dense)            (None, 20)                620       
                                                                 
 dense_26 (Dense)            (None, 10)                210       
                                                                 
 dense_27 (Dense)            (None, 5)                 55        
                                                                 
 dense_28 (Dense)            (None, 10)                60        
                                                                 
 dense_29 (Dense)            (None, 20)                220 

In [29]:
ecv_dae = DAE_reduction(ecv)

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 756)]             0         
                                                                 
 dense_32 (Dense)            (None, 30)                22710     
                                                                 
 dense_33 (Dense)            (None, 20)                620       
                                                                 
 dense_34 (Dense)            (None, 10)                210       
                                                                 
 dense_35 (Dense)            (None, 5)                 55        
                                                                 
 dense_36 (Dense)            (None, 10)                60        
                                                                 
 dense_37 (Dense)            (None, 20)                220 

In [30]:
ff_dae = DAE_reduction(ff)

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 465)]             0         
                                                                 
 dense_40 (Dense)            (None, 30)                13980     
                                                                 
 dense_41 (Dense)            (None, 20)                620       
                                                                 
 dense_42 (Dense)            (None, 10)                210       
                                                                 
 dense_43 (Dense)            (None, 5)                 55        
                                                                 
 dense_44 (Dense)            (None, 10)                60        
                                                                 
 dense_45 (Dense)            (None, 20)                220

In [32]:
np.savetxt("extracted_features/bank_reserves_DAE.csv", br_dae, delimiter=",")
np.savetxt("extracted_features/epstein_DAE.csv", ecv_dae, delimiter=",")
np.savetxt("extracted_features/ff_DAE.csv", ff_dae, delimiter=",")